# Pràctica 4 PLH - Rubén Álvarez Aragonés i Pol Pérez Prades

____

# Stuff

## Imports 

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf
import numpy as np
import nltk
import re
import spacy
from spacy.lang.ca.examples import sentences 
from gensim.models import word2vec
import torch
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Requisites
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np


nltk.download('punkt')
nltk.download('stopwords')

spacy.cli.download("ca_core_news_sm")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ralva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ralva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


## Entrenament model Word2Vec

In [8]:
class Word2VecEmbedder:
    def __init__(self, corpus_path, corpus_size, load_model=False, model_path=None):
        if not load_model:
            self.corpus_path = corpus_path
            self.corpus_size = int(corpus_size * 2**30)  # Convert GB to bytes
            self.corpus = self.get_corpus(corpus_path)
            self.fit()
        else:
            try:
                self.load(model_path)
            except FileNotFoundError:
                print("Model not found. Please check the path.")
                return

    def get_corpus(self, corpus_path):
        with open(corpus_path, 'r', encoding='utf-8') as f:
            print("Reading corpus...")
            corpus = f.read(self.corpus_size)
            print("Preprocessing corpus...")
            corpus = self.preprocess(corpus)  # Preprocess the corpus and tokenize it
        return corpus

    def fit(self, window_size=15, vector_size=300, min_count=10, workers=8, epochs=10):
        # Initialize the Word2Vec model with gensim
        print("Initializing Word2Vec model...")
        self.model = word2vec.Word2Vec(sentences=[self.corpus], vector_size=vector_size, window=window_size, min_count=min_count, workers=workers, epochs=epochs)
        print("Model training completed.")

    def save(self, model_path):
        # Save the model
        self.model.save(model_path)

    def load(self, model_path):
        # Load the model
        self.model = word2vec.Word2Vec.load(model_path)

    def preprocess(self, corpus):
        # Lowercase the corpus
        print("Lowercasing...")
        corpus = corpus.lower()
        
        # Remove special characters
        print("Removing special characters...")
        corpus = re.sub(r'[^a-záàéèíìóòúùñüç\s]', ' ', corpus)
        
        # Tokenize the corpus
        print("Tokenizing...")
        corpus = nltk.word_tokenize(corpus)
        
        # Eliminate last token (probably incomplete word)
        corpus = corpus[:-1]
        
        return corpus

    def get_embedding(self, word):
        # Get the embedding of a word
        try:
            return self.model.wv[word]
        except KeyError:
            print(f"Word '{word}' not in vocabulary.")
            return None

    def print_vocab(self):
        print("Vocabulary:", list(self.model.wv.index_to_key))
        
    

### 1. Model amb 100MB de dades

In [9]:
Word2Vec_model = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 0.1)

Reading corpus...
Preprocessing corpus...
Lowercasing...
Removing special characters...
Tokenizing...
Initializing Word2Vec model...
Model training completed.


In [ ]:
Word2Vec_model.save('models/word2vec_model.bin')

In [ ]:
Word2Vec_model.print_vocab()

Vocabulary: ['de', 'la', 'i', 'a', 'que', 'el', 'l', 'en', 'd', 'per', 'els', 'del', 'les', 'un', 'amb', 'una', 'va', 'ha', 'no', 'al', 'es', 'és', 'més', 'com', 'dels', 's', 'aquest', 'hi', 'si', 'o', 'se', 'tot', 'aquesta', 'han', 'fer', 'molt', 'però', 'ser', 'també', 'ja', 'li', 'seu', 'als', 'seva', 'n', 'cap', 'seus', 't', 'ens', 'pàgina', 'veure', 'sobre', 'catalunya', 'pel', 'després', 'estat', 'ho', 'perquè', 'varen', 'on', 'entre', 'fet', 'et', 'català', 'havia', 'lleida', 'tots', 'quan', 'qui', 'són', 'ara', 'm', 'van', 'des', 'fins', 'hem', 'això', 'era', 'fa', 'partit', 'dia', 'anys', 'dir', 'sempre', 'diputació', 'barcelona', 'què', 'part', 'altres', 'només', 'contra', 'durant', 'dos', 'sense', 'equip', 'any', 'voltants', 'interessa', 'recomanem', 'informació', 'primer', 'em', 'col', 'visitis', 'lloc', 'gran', 'hitler', 'tant', 'restaurant', 'cada', 'ni', 'comen', 'president', 'encara', 'persones', 'e', 'té', 'així', 'ar', 'estan', 'pot', 'està', 'llengua', 'mateix', 'vam

In [10]:
Word2Vec_model.get_embedding("de")

array([-1.05623817e+00,  1.16298509e+00,  1.66580435e-02,  4.87576127e-01,
        3.13172899e-02, -1.82584894e+00, -2.30049655e-01,  2.70141053e+00,
       -4.84138817e-01, -3.24209891e-02,  1.37779248e+00, -1.44781351e+00,
       -9.86493409e-01,  1.05396032e+00, -5.43474913e-01, -3.20039354e-02,
        2.31937456e+00,  8.36090967e-02,  5.18369853e-01, -1.09011912e+00,
       -5.41573465e-01, -3.54912803e-02,  4.68922943e-01,  9.67941463e-01,
        1.15948009e+00, -9.18916315e-02, -3.11388522e-01,  7.30274737e-01,
       -1.53097129e+00, -2.08533382e+00,  9.86677885e-01, -2.47951001e-01,
       -1.70090288e-01, -8.44896972e-01, -2.47684911e-01,  1.46598876e+00,
        4.04482365e-01, -2.00736976e+00,  7.34920055e-02, -1.14428222e+00,
        6.67930171e-02, -6.38586223e-01,  8.04345727e-01, -5.70785165e-01,
        1.89507866e+00, -8.47234298e-03, -1.51646048e-01,  8.65665734e-01,
       -3.80184889e-01, -6.16664700e-02, -3.21577638e-02,  2.01616430e+00,
       -2.61370867e-01,  

In [14]:
Word2Vec_model.model.wv.most_similar("negre")

[('ensems', 0.27725985646247864),
 ('profètic', 0.24925659596920013),
 ('epigenètica', 0.23335236310958862),
 ('federatius', 0.23306740820407867),
 ('sèquia', 0.2311156988143921),
 ('consagrat', 0.22697941958904266),
 ('èric', 0.22261522710323334),
 ('emprant', 0.218054860830307),
 ('allà', 0.2145397961139679),
 ('eliminatòries', 0.21094447374343872)]

### 2. Model amb 500MB de dades

In [13]:
Word2Vec_model_500 = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 0.5)

Reading corpus...
Preprocessing corpus...
Lowercasing...
Removing special characters...
Tokenizing...
Initializing Word2Vec model...
Model training completed.


In [15]:
Word2Vec_model.get_embedding("hola")

array([ 2.0775015e-03,  1.0567880e-03, -1.6255629e-03,  3.1482291e-03,
       -1.0828007e-03,  8.4958872e-04,  3.2786091e-03, -2.7655109e-03,
        3.2775463e-03,  6.6130323e-04,  1.2716241e-03,  1.1530495e-03,
       -4.9383880e-04,  1.9930000e-03, -1.8229814e-03,  4.5082212e-04,
        2.7142679e-03, -2.6555145e-03, -7.3076884e-04,  3.1427550e-03,
       -2.6453920e-03, -3.0769189e-03,  3.1929337e-03,  3.1773886e-03,
        2.0901405e-03, -3.0516200e-03,  2.4018362e-03,  1.8233713e-03,
       -2.6497131e-03, -2.0127888e-03, -8.6779514e-04,  2.7872846e-03,
        3.2888358e-03, -1.3961394e-04,  5.7863514e-04,  2.0927347e-03,
       -3.2404303e-03, -2.4493536e-04,  1.2174038e-03,  3.0606557e-03,
        1.9006491e-04,  2.1212928e-03, -8.0346031e-04, -1.3436433e-03,
        8.4563333e-04, -2.5548593e-03, -2.0761208e-03,  2.9200499e-03,
       -2.6264484e-03, -1.8010263e-03,  1.2320471e-03,  2.3446269e-03,
        1.1889736e-04,  2.8769842e-03, -2.2959744e-03,  2.7552882e-04,
      

In [22]:
Word2Vec_model_500.model.wv.most_similar("inshalla")

KeyError: "Key 'inshalla' not present in vocabulary"

### 3. Model amb 1GB de dades

In [ ]:
Word2Vec_model_1024 = Word2VecEmbedder('corpus\catalan_general_crawling.txt', 1)

Reading corpus...
Preprocessing corpus...
Lowercasing...
Removing special characters...
Tokenizing...
Initializing Word2Vec model...
Model training completed.


In [ ]:
Word2Vec_model.get_embedding("hola")

array([-7.9389382e-03, -8.9844353e-03, -4.5826603e-03,  5.5299234e-03,
       -9.4605685e-04,  1.1343563e-03, -1.9249201e-04,  4.0157423e-03,
        6.8704914e-03, -7.0510614e-03,  5.8637308e-03,  9.9222707e-03,
       -7.2787488e-03, -6.7374315e-03, -1.8864155e-03, -5.3431201e-03,
        9.9086938e-03, -4.0579177e-03, -1.2240529e-03,  2.8516566e-03,
       -4.5498814e-03, -1.2679828e-03, -4.3294239e-03, -8.5449358e-03,
       -3.0690837e-03, -6.9884383e-03, -6.1625205e-03, -9.2972824e-03,
       -5.3252899e-03,  3.2473837e-03,  3.1876445e-03, -7.0571471e-03,
        1.5914536e-03,  2.7031647e-03, -2.1402992e-03,  3.0083584e-03,
       -5.5540358e-03, -3.5628723e-03,  7.7224839e-03, -2.8472329e-03,
       -8.2702264e-03, -4.8768856e-03,  1.6037774e-03, -2.0911193e-03,
       -7.4587557e-03, -5.7342900e-03, -3.9162696e-03,  2.6814414e-03,
        6.2334309e-03,  1.3791680e-03, -1.8893396e-03,  8.2629528e-03,
       -9.6584987e-03, -2.8164326e-03, -7.6088239e-03,  9.0056704e-03,
      

## Entrenament model de Similitud de Text Semàntic

### Imports

In [1]:
from importació_data import read_all_ts_data, reformat_data, create_corpus, preprocess, flattened_corpus_count

In [2]:
train, test, val = read_all_ts_data()
train, test, val = reformat_data(train, test, val)
corpus, all_words = create_corpus(train, test, val, preprocess=preprocess)
flat_corpus = flattened_corpus_count(corpus)

### Compartació amb diferents models de Word Embeddings

#### 1. One Hot

In [3]:
from onehot import map_one_hot
from importació_data import pair_list_to_x_y
from model_bàsic import build_and_compile_model_better
import tensorflow as tf

In [4]:
# Eliminate from all_words, the words that are repeated very few times or too much times
keys_preprocess = [index  for index in all_words if flat_corpus[index] > 10 and flat_corpus[index] < 50]

In [5]:
mapped_one_hot_train = map_one_hot(train, all_words, keys_preprocess)
mapped_one_hot_test = map_one_hot(test, all_words, keys_preprocess)
mapped_one_hot_val = map_one_hot(val, all_words, keys_preprocess)

In [6]:
x_train, y_train = pair_list_to_x_y(mapped_one_hot_train)
x_val, y_val = pair_list_to_x_y(mapped_one_hot_val)

In [7]:
batch_size: int = 64
num_epochs: int = 64

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [9]:
x_train[0].shape, x_train[1].shape, y_train.shape

((2043, 806), (2043, 806), (2043,))

In [10]:
embedding_size = len(keys_preprocess)
model = build_and_compile_model_better(embedding_size = embedding_size)
#tf.keras.utils.plot_model(model, show_shapes=True, show_layer_activations=True, )
print(model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 806)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 806)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 806)       │    650,442 │ input_layer[0][0… │
│                     │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 806)       │          0 │ dense[0][0],      │
│                     │                   │            │ dense[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 806)       │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │     12,912 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 16)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         17 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 1)         │          0 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 663,371 (2.53 MB)

 Trainable params: 663,371 (2.53 MB)

 Non-trainable params: 0 (0.00 B)

None


In [11]:
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Epoch 1/64


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.7273 - val_loss: 0.7472
Epoch 2/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.7165 - val_loss: 0.7434
Epoch 3/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.7102 - val_loss: 0.7405
Epoch 4/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.6976 - val_loss: 0.7362
Epoch 5/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.6698 - val_loss: 0.7280
Epoch 6/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.6050 - val_loss: 0.7258
Epoch 7/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.4980 - val_loss: 0.7436
Epoch 8/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.3817 - val_loss: 0.7721
Epoch 9/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2925 - val_loss: 0.7940
Epoch 10/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2430 - val_loss: 0.8144
Epoch 11/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2052 - val_loss: 0.8209
Epoch 12/64
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1831 - val_l

In [12]:
from model_bàsic import compute_pearson

x_test, y_test = pair_list_to_x_y(mapped_one_hot_test)

In [15]:
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson (train): {compute_pearson(model, x_train, y_train)}")
print(f"Correlación de Pearson (validation): {compute_pearson(model, x_val, y_val)}")
print(f"Correlación de Pearson (test): {compute_pearson(model, x_test, y_test)}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Correlación de Pearson (train): 0.9615968429891668
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Correlación de Pearson (validation): 0.18542286227091276
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Correlación de Pearson (test): 0.18361106295459315


#### 2. Word2Vec preentrenats 

#### 3. SpaCy

In [ ]:
nlp = spacy.load("en_core_web_md")
sentence = nlp("I sit on a bank.")
sentence[4].vector
# -> NDArray

#### 4. RoBERTa 

In [ ]:
nlp = spacy.load("ca_core_news_trf")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

#### 5. RoBERTa fine-tuned

In [ ]:
model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

sentence_pairs = [("El llibre va caure per la finestra.", "El llibre va sortir volant."),
                  ("M'agrades.", "T'estimo."),
                  ("M'agrada el sol i la calor", "A la Garrotxa plou molt.")]

predictions = pipe(prepare(sentence_pairs), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    prediction['score'] = logit(prediction['score'])
print(predictions)